# Segmentation and clustering 

Importing  Liberaries

In [1]:
import pandas as pd
import numpy as np
import xlrd
print ("Liberaries imprted!")

Liberaries imprted!


# Solving Part 1

First we scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 


Second, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [10]:
df=pd.read_excel('C:/Users/r/Desktop/projects/Coursera_Capstone/toronto.xlsx', Sheet='Sheet1')
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Removingd dublicates pincode and Not assigned Neighbourhood

In [11]:
df2=df[df['Neighbourhood'] != 'Not assigned']
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


combining rows having pincodes

In [65]:
df2=df2.groupby(['Postcode','Borough' ]).agg(lambda x: ",".join(x)).reset_index()
df2.head(11)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [64]:
df2.shape

(103, 3)

# Testing different method of importing geo coder library 

In [69]:
import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'sensor': 'false', 'address': 'Mountain View, CA'}
r = requests.get(url, params=params)
results = r.json()['results']
location = results[0]['geometry']['location']
location['lat'], location['lng']

IndexError: list index out of range

In [78]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code='M5G'
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]


KeyboardInterrupt: 

In [73]:
from geopy.geocoders import Nominatim
def get_geo_info(place_name):
   
    

    # Create geo_locator object instance
    geo_locator = Nominatim()

    # Attempt to obtain geo data for given place name
    try:
        location = geo_locator.geocode(place_name)
    except Exception:
        raise Exception("Location error")

    if not location:
        raise Exception("Location error")

    return location 

In [75]:
get_geo_info('New York, NY')

c:\users\r\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  import sys


Location(New York, United States of America, (40.7127281, -74.0060152, 0.0))

# Solving Part 2

### loading geo coordinates of Toronto City

In [80]:
g_code=pd.read_csv('Geospatial_Coordinates.csv')

In [83]:
g_code.rename(columns={'Postal Code':'Postcode'}, inplace=True)
g_code.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [88]:
df3=pd.merge(df2,g_code, how='inner', on=['Postcode'])
print('New Dataframe after merging geo coordinates')
df3.head()


New Dataframe after merging geo coordinates


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [90]:
print('Size of New Dataframe :-')
df3.shape

Size of New Dataframe :-


(103, 5)